In [206]:
import pandas as pd
import numpy as np

import copy
import os
import re

# Erroneous input (two policies ranked the same) is resolved randomly.
# Set the seed to prevent flaky voting.
np.random.seed(42)

In [3]:
working_dir = '/home/rgiordan/Documents/git_repos/Presentations/CCC_covid_vote'

In [7]:
responses = pd.read_csv(os.path.join(working_dir, 'example.csv'))

In [112]:
# Select the columns that compile the votes

#print(responses.columns)
a_col = responses.columns[3]
b_col = responses.columns[4]
c_col = responses.columns[5]
d_col = responses.columns[6]
# print('\n'.join([a_col, b_col, c_col, d_col]))

# Assert that I have selected the columns correctly
assert(re.search("Policy A", a_col))
assert(re.search("Policy B", b_col))
assert(re.search("Policy C", c_col))
assert(re.search("Policy D", d_col))

In [197]:
# Load the CSV and convert the votes to numeric ranks.

def ConvertToNumericVote(pd_col):
    # Empty rows (not voted on) are storted as nan, not strings.
    vote_str = pd_col.fillna('0').to_numpy()
    
    # Keep only the numbers.
    vote_str = np.array([ re.sub(r'[^0-9]', '', v) for v in vote_str ])
    
    return vote_str.astype('int')

votes = np.vstack([
    ConvertToNumericVote(responses[col]) for col in [ a_col, b_col, c_col, d_col ]
])
print(votes)

# Assert that the A vote is the first row as expected.
assert(np.all(ConvertToNumericVote(responses[a_col]) == votes[0, :]))

[[2 4 4 4 0 4 3 4 4 4 4 2 4 4 3 2 4 3 2]
 [3 3 3 3 0 2 4 3 3 2 1 1 3 2 4 1 3 4 3]
 [1 2 2 2 2 1 2 2 2 3 2 3 2 3 3 3 2 2 1]
 [4 1 1 1 1 3 1 1 1 1 3 4 1 1 1 4 1 1 4]]


array([0, 1, 2])

In [226]:
# Check for valid input and correct mistakes according to the office hours document.
#vote = votes[:, 0]
vote = np.array([0, 2, 2, 4])

print(vote)

inds = vote > 0
num_votes = sum(inds)
print(num_votes)
rank = 1
while rank <= num_votes:
    print(f'Rank {rank}')
    if not np.any(vote == rank):
        # There is nothing with this rank; decrement the other votes and try again.
        print('Decrementing')
        vote[inds] = vote[inds] - 1
    else:
        rank_inds = np.argwhere(vote == rank).flatten()
        if len(rank_inds) > 1:
            # This rank was duplicated.  Randomly split up this rank among the
            # policies.
            print(f'Duplicating ({rank_inds})')

            num_dups = len(rank_inds)
            new_ranks = np.random.choice(num_dups, num_dups, replace=False) + rank

            # Decrement other votes to make room
            dec_inds = np.logical_and(vote < rank + num_dups - 1, inds) 
            vote[dec_inds] = vote[dec_inds] - (num_dups - 1)
            vote[rank_inds] = new_ranks
            rank += num_dups
        else:
            # This rank is okay, go to the next one.
            rank += 1
    print(vote)
            
print(vote)

[0 2 2 4]
Rank 1
Decrementing
[0 1 1 3]
Rank 1
Duplicating ([1 2])
[0 2 1 3]
Rank 3
[0 2 1 3]
Rank 4
Decrementing
[0 1 0 2]
Rank 4
Decrementing
[ 0  0 -1  1]
Rank 4
Decrementing
[ 0 -1 -2  0]
Rank 4
Decrementing
[ 0 -2 -3 -1]
Rank 4
Decrementing
[ 0 -3 -4 -2]
Rank 4
Decrementing
[ 0 -4 -5 -3]
Rank 4
Decrementing
[ 0 -5 -6 -4]
Rank 4
Decrementing
[ 0 -6 -7 -5]
Rank 4
Decrementing
[ 0 -7 -8 -6]
Rank 4
Decrementing
[ 0 -8 -9 -7]
Rank 4
Decrementing
[  0  -9 -10  -8]
Rank 4
Decrementing
[  0 -10 -11  -9]
Rank 4
Decrementing
[  0 -11 -12 -10]
Rank 4
Decrementing
[  0 -12 -13 -11]
Rank 4
Decrementing
[  0 -13 -14 -12]
Rank 4
Decrementing
[  0 -14 -15 -13]
Rank 4
Decrementing
[  0 -15 -16 -14]
Rank 4
Decrementing
[  0 -16 -17 -15]
Rank 4
Decrementing
[  0 -17 -18 -16]
Rank 4
Decrementing
[  0 -18 -19 -17]
Rank 4
Decrementing
[  0 -19 -20 -18]
Rank 4
Decrementing
[  0 -20 -21 -19]
Rank 4
Decrementing
[  0 -21 -22 -20]
Rank 4
Decrementing
[  0 -22 -23 -21]
Rank 4
Decrementing
[  0 -23 -24 -22]


Decrementing
[   0 -607 -608 -606]
Rank 4
Decrementing
[   0 -608 -609 -607]
Rank 4
Decrementing
[   0 -609 -610 -608]
Rank 4
Decrementing
[   0 -610 -611 -609]
Rank 4
Decrementing
[   0 -611 -612 -610]
Rank 4
Decrementing
[   0 -612 -613 -611]
Rank 4
Decrementing
[   0 -613 -614 -612]
Rank 4
Decrementing
[   0 -614 -615 -613]
Rank 4
Decrementing
[   0 -615 -616 -614]
Rank 4
Decrementing
[   0 -616 -617 -615]
Rank 4
Decrementing
[   0 -617 -618 -616]
Rank 4
Decrementing
[   0 -618 -619 -617]
Rank 4
Decrementing
[   0 -619 -620 -618]
Rank 4
Decrementing
[   0 -620 -621 -619]
Rank 4
Decrementing
[   0 -621 -622 -620]
Rank 4
Decrementing
[   0 -622 -623 -621]
Rank 4
Decrementing
[   0 -623 -624 -622]
Rank 4
Decrementing
[   0 -624 -625 -623]
Rank 4
Decrementing
[   0 -625 -626 -624]
Rank 4
Decrementing
[   0 -626 -627 -625]
Rank 4
Decrementing
[   0 -627 -628 -626]
Rank 4
Decrementing
[   0 -628 -629 -627]
Rank 4
Decrementing
[   0 -629 -630 -628]
Rank 4
Decrementing
[   0 -630 -631 -629]

[    0 -1271 -1272 -1270]
Rank 4
Decrementing
[    0 -1272 -1273 -1271]
Rank 4
Decrementing
[    0 -1273 -1274 -1272]
Rank 4
Decrementing
[    0 -1274 -1275 -1273]
Rank 4
Decrementing
[    0 -1275 -1276 -1274]
Rank 4
Decrementing
[    0 -1276 -1277 -1275]
Rank 4
Decrementing
[    0 -1277 -1278 -1276]
Rank 4
Decrementing
[    0 -1278 -1279 -1277]
Rank 4
Decrementing
[    0 -1279 -1280 -1278]
Rank 4
Decrementing
[    0 -1280 -1281 -1279]
Rank 4
Decrementing
[    0 -1281 -1282 -1280]
Rank 4
Decrementing
[    0 -1282 -1283 -1281]
Rank 4
Decrementing
[    0 -1283 -1284 -1282]
Rank 4
Decrementing
[    0 -1284 -1285 -1283]
Rank 4
Decrementing
[    0 -1285 -1286 -1284]
Rank 4
Decrementing
[    0 -1286 -1287 -1285]
Rank 4
Decrementing
[    0 -1287 -1288 -1286]
Rank 4
Decrementing
[    0 -1288 -1289 -1287]
Rank 4
Decrementing
[    0 -1289 -1290 -1288]
Rank 4
Decrementing
[    0 -1290 -1291 -1289]
Rank 4
Decrementing
[    0 -1291 -1292 -1290]
Rank 4
Decrementing
[    0 -1292 -1293 -1291]
Rank 4
D

[    0 -1904 -1905 -1903]
Rank 4
Decrementing
[    0 -1905 -1906 -1904]
Rank 4
Decrementing
[    0 -1906 -1907 -1905]
Rank 4
Decrementing
[    0 -1907 -1908 -1906]
Rank 4
Decrementing
[    0 -1908 -1909 -1907]
Rank 4
Decrementing
[    0 -1909 -1910 -1908]
Rank 4
Decrementing
[    0 -1910 -1911 -1909]
Rank 4
Decrementing
[    0 -1911 -1912 -1910]
Rank 4
Decrementing
[    0 -1912 -1913 -1911]
Rank 4
Decrementing
[    0 -1913 -1914 -1912]
Rank 4
Decrementing
[    0 -1914 -1915 -1913]
Rank 4
Decrementing
[    0 -1915 -1916 -1914]
Rank 4
Decrementing
[    0 -1916 -1917 -1915]
Rank 4
Decrementing
[    0 -1917 -1918 -1916]
Rank 4
Decrementing
[    0 -1918 -1919 -1917]
Rank 4
Decrementing
[    0 -1919 -1920 -1918]
Rank 4
Decrementing
[    0 -1920 -1921 -1919]
Rank 4
Decrementing
[    0 -1921 -1922 -1920]
Rank 4
Decrementing
[    0 -1922 -1923 -1921]
Rank 4
Decrementing
[    0 -1923 -1924 -1922]
Rank 4
Decrementing
[    0 -1924 -1925 -1923]
Rank 4
Decrementing
[    0 -1925 -1926 -1924]
Rank 4
D

[    0 -2571 -2572 -2570]
Rank 4
Decrementing
[    0 -2572 -2573 -2571]
Rank 4
Decrementing
[    0 -2573 -2574 -2572]
Rank 4
Decrementing
[    0 -2574 -2575 -2573]
Rank 4
Decrementing
[    0 -2575 -2576 -2574]
Rank 4
Decrementing
[    0 -2576 -2577 -2575]
Rank 4
Decrementing
[    0 -2577 -2578 -2576]
Rank 4
Decrementing
[    0 -2578 -2579 -2577]
Rank 4
Decrementing
[    0 -2579 -2580 -2578]
Rank 4
Decrementing
[    0 -2580 -2581 -2579]
Rank 4
Decrementing
[    0 -2581 -2582 -2580]
Rank 4
Decrementing
[    0 -2582 -2583 -2581]
Rank 4
Decrementing
[    0 -2583 -2584 -2582]
Rank 4
Decrementing
[    0 -2584 -2585 -2583]
Rank 4
Decrementing
[    0 -2585 -2586 -2584]
Rank 4
Decrementing
[    0 -2586 -2587 -2585]
Rank 4
Decrementing
[    0 -2587 -2588 -2586]
Rank 4
Decrementing
[    0 -2588 -2589 -2587]
Rank 4
Decrementing
[    0 -2589 -2590 -2588]
Rank 4
Decrementing
[    0 -2590 -2591 -2589]
Rank 4
Decrementing
[    0 -2591 -2592 -2590]
Rank 4
Decrementing
[    0 -2592 -2593 -2591]
Rank 4
D

[    0 -3261 -3262 -3260]
Rank 4
Decrementing
[    0 -3262 -3263 -3261]
Rank 4
Decrementing
[    0 -3263 -3264 -3262]
Rank 4
Decrementing
[    0 -3264 -3265 -3263]
Rank 4
Decrementing
[    0 -3265 -3266 -3264]
Rank 4
Decrementing
[    0 -3266 -3267 -3265]
Rank 4
Decrementing
[    0 -3267 -3268 -3266]
Rank 4
Decrementing
[    0 -3268 -3269 -3267]
Rank 4
Decrementing
[    0 -3269 -3270 -3268]
Rank 4
Decrementing
[    0 -3270 -3271 -3269]
Rank 4
Decrementing
[    0 -3271 -3272 -3270]
Rank 4
Decrementing
[    0 -3272 -3273 -3271]
Rank 4
Decrementing
[    0 -3273 -3274 -3272]
Rank 4
Decrementing
[    0 -3274 -3275 -3273]
Rank 4
Decrementing
[    0 -3275 -3276 -3274]
Rank 4
Decrementing
[    0 -3276 -3277 -3275]
Rank 4
Decrementing
[    0 -3277 -3278 -3276]
Rank 4
Decrementing
[    0 -3278 -3279 -3277]
Rank 4
Decrementing
[    0 -3279 -3280 -3278]
Rank 4
Decrementing
[    0 -3280 -3281 -3279]
Rank 4
Decrementing
[    0 -3281 -3282 -3280]
Rank 4
Decrementing
[    0 -3282 -3283 -3281]
Rank 4
D

KeyboardInterrupt: 

In [188]:
def FindWinnerAndLoser(votes):
    num_voters = votes.shape[1]
    num_votes = np.sum(votes == 1)
    assert(num_votes <= num_voters)
    if num_votes != num_voters:
        print('Some did not vote. ', num_voters - num_votes)
    vote_count = np.sum(votes == 1, axis=1)
    loser = np.argmin(vote_count)
    majority = vote_count / num_votes >= 0.5
    if np.any(majority):
        winner = np.argwhere(majority)[0]
        print(winner)
        if (len(winner) > 1):
            print('There was a tie')
        print(len(winner))
        return (winner, loser)
    else:
        print('No majority')
        return (-1, loser)
    
def RemovePolicy(votes, drop_index):
    new_votes = copy.copy(votes)
    drop_index = 3
    increment_cols = np.argwhere(new_votes[drop_index, :] == 1).flatten()
    new_votes[:, increment_cols] = votes[:, increment_cols] - 1
    new_votes[new_votes < 0] = 0
    new_votes[drop_index, :] = 0
    return new_votes

print('\nOriginal:')
FindWinnerAndLoser(votes)

print('\nThree stripped:')
FindWinnerAndLoser(RemovePolicy(votes, 3))


Original:
[3]
1

Three stripped:
Some did not vote.  1
[2]
1


(array([2]), 0)

In [191]:
votes
print(np.sum(votes == 1))
print(np.sum(RemovePolicy(votes, 3) == 1))

19
18


In [193]:
print(votes)

print(RemovePolicy(votes, 3))

[[2 4 4 4 0 4 3 4 4 4 4 2 4 4 3 2 4 3 2]
 [3 3 3 3 0 2 4 3 3 2 1 1 3 2 4 1 3 4 3]
 [1 2 2 2 2 1 2 2 2 3 2 3 2 3 3 3 2 2 1]
 [4 1 1 1 1 3 1 1 1 1 3 4 1 1 1 4 1 1 4]]
[[2 3 3 3 0 4 2 3 3 3 4 2 3 3 2 2 3 2 2]
 [3 2 2 2 0 2 3 2 2 1 1 1 2 1 3 1 2 3 3]
 [1 1 1 1 1 1 1 1 1 2 2 3 1 2 2 3 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
